In [ ]:
'''Code for feature extraction'''

In [7]:
'''Importing preprocessed data(csv files) into DataFrame of pandas'''

import pandas as pd
df=pd.read_excel(r"C:\Users\Bhavya Prakash\OneDrive\Desktop\INMAS, DRDO\eeg-during-mental-arithmetic-tasks-1.0.0\pre_processed data (CSV files)\Subject_00_1.xlsx")
df

,EEG Fp1,EEG Fp2,EEG F3,EEG F4,EEG F7,EEG F8,EEG T3,EEG T4,EEG C3,EEG C4,...,EEG T6,EEG P3,EEG P4,EEG O1,EEG O2,EEG Fz,EEG Cz,EEG Pz,EEG A2-A1,ECG ECG
0,1.404490e-06,1.432121e-06,1.648694e-06,1.258935e-06,8.636038e-07,1.117674e-06,-2.615847e-07,-5.924262e-07,1.878112e-07,2.753330e-07,...,-1.660840e-06,-1.467584e-06,-1.639625e-06,-1.339082e-06,-1.407805e-06,1.221600e-06,-4.362171e-06,-2.230830e-06,1.179448e-06,4.780171e-06
1,-1.691623e-06,-1.720957e-06,-1.992834e-06,-1.519862e-06,-1.039618e-06,-1.336644e-06,3.200685e-07,7.182113e-07,-2.281121e-07,-3.322665e-07,...,2.007082e-06,1.772315e-06,1.982589e-06,1.630478e-06,1.707245e-06,-1.475533e-06,5.237511e-06,2.688023e-06,-1.417522e-06,-5.809042e-06
2,-4.429381e-06,-4.293336e-06,-5.606788e-06,-4.175330e-06,-2.706807e-06,-2.958050e-06,1.131496e-06,2.146187e-06,-6.945833e-07,-9.027776e-07,...,5.641036e-06,4.905914e-06,5.625088e-06,5.278063e-06,5.150206e-06,-4.093055e-06,1.276797e-05,7.110311e-06,-3.539768e-06,-1.819755e-05
3,-6.213604e-06,-5.543466e-06,-8.784628e-06,-6.329166e-06,-3.863768e-06,-2.751952e-06,2.428316e-06,3.707136e-06,-1.145528e-06,-1.339515e-06,...,8.971279e-06,7.567761e-06,8.971192e-06,1.008464e-05,9.137147e-06,-6.280739e-06,1.543314e-05,1.024279e-05,-4.539165e-06,-3.380271e-05
4,-6.607037e-06,-5.310364e-06,-1.057104e-05,-7.432817e-06,-4.384243e-06,-9.136795e-07,4.125808e-06,5.018860e-06,-1.282470e-06,-1.630070e-06,...,1.139198e-05,9.138046e-06,1.106762e-05,1.498316e-05,1.297982e-05,-7.418417e-06,1.292355e-05,1.162480e-05,-4.238317e-06,-5.032446e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46587,2.007149e-09,6.905925e-09,1.028683e-09,-7.034218e-10,2.677232e-09,2.854031e-09,6.037918e-09,1.119554e-09,1.435469e-09,1.243192e-10,...,2.257042e-09,3.376429e-09,3.351960e-09,1.471922e-09,3.080207e-09,-7.296854e-10,2.665010e-09,3.691291e-09,3.338180e-09,-4.873623e-08
46588,1.648944e-09,5.676774e-09,9.485400e-10,-2.550811e-10,1.902209e-09,2.254104e-09,4.784232e-09,9.886193e-10,1.278791e-09,3.361787e-10,...,1.705653e-09,2.605195e-09,2.728506e-09,1.004970e-09,2.430019e-09,-2.719580e-10,2.357240e-09,2.860574e-09,2.359559e-09,-3.931223e-08
46589,1.207757e-09,4.147690e-09,5.838170e-10,-4.524327e-10,1.698111e-09,1.774460e-09,3.729135e-09,6.690934e-10,8.407955e-10,3.481311e-11,...,1.383321e-09,2.072449e-09,1.998124e-09,9.147948e-10,1.849848e-09,-4.731534e-10,1.522477e-09,2.219235e-09,2.066440e-09,-2.953441e-08
46590,8.450009e-10,2.913775e-09,5.017915e-10,-2.042211e-11,9.141699e-10,1.150806e-09,2.425685e-09,5.370044e-10,6.682913e-10,2.385787e-10,...,8.441603e-10,1.279588e-09,1.376698e-09,4.408874e-10,1.214417e-09,-3.250557e-11,1.232585e-09,1.393233e-09,1.101569e-09,-1.997063e-08


In [1]:
'''Code for plotting of EEG channels for reference
    Plotting of PSDs
    Calculating theta/aplha bandpower'''

# Importing libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.2)
import numpy as np
from scipy import signal
from scipy.integrate import simps


# Defining x-axis values that will be time
x=np.linspace(1,60,num=46592)
print(len(x))


# Defining y-axis values that will be volatges
y=df["EEG Fp1"]


# Defining sampling frequency as 256 Hz
sf = 256
# time = np.arange(df.size) / sf
time=x


# Plotting EEG signal from "Fp1" channel
fig, ax = plt.subplots(1, 1, figsize=(22, 8))
plt.plot(time, df["EEG Fp1"], lw=1.5, color='k')
plt.xlabel('Time (seconds)')
plt.ylabel('Voltage')
plt.xlim([time.min(), time.max()])
plt.title('EEG Fp1')
sns.despine()


# Plotting of Power Spectral Density
# Define window length (4 seconds)
win = 4 * sf
freqs, psd = signal.welch(df["EEG Fp1"], sf, nperseg=win)
sns.set(font_scale=1.2, style='white')
plt.figure(figsize=(8, 4))
plt.plot(freqs, psd, color='k', lw=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (V^2 / Hz)')
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram")
plt.xlim([0, freqs.max()])
sns.despine()


# Declaring of each wave(delta / theta / alpha / beta / gamma) in the PSD
# Declaring area of delta waves
low, high = 0.5, 4
# Find intersecting values in frequency vector
idx_delta = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_delta, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 10])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for delta waves")
sns.despine()


# Declaring area of theta waves
low, high = 4,8
# Find intersecting values in frequency vector
idx_theta = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_theta, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 10])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for theta waves")
sns.despine()


# Declaring area of alpha waves
low, high = 8,12
# Find intersecting values in frequency vector
idx_alpha = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_alpha, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 16])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for alpha waves")
sns.despine()


# Declaring area of beta waves
low, high = 12,30
# Find intersecting values in frequency vector
idx_beta = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_beta, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 34])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for beta waves")
sns.despine()


# Declaring area of gamma waves
low, high = 30,45
# Find intersecting values in frequency vector
idx_gamma = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_gamma, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 50])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for gamma waves")
sns.despine()


# computing the absolute power of each wave
# Frequency resolution
freq_res = freqs[1] - freqs[0]  # = 1 / 4 = 0.25
# Compute the absolute power by approximating the area under the curve
delta_power = simps(psd[idx_delta], dx=freq_res)
print('Absolute delta power: %.12f uV^2' % delta_power)


# Computing realtive power of each wave
from scipy.integrate import simps
total_power = simps(psd, dx=freq_res)
delta_rel_power = delta_power / total_power
print('Relative delta power: %.3f' % delta_rel_power)


# Defining a function to calculate bandpower of each wave
def bandpower(data, sf, band, window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
        If None, window_sec = (1 / min(band)) * 2
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    band = np.asarray(band)
    low, high = band
    # Define window length
    if window_sec is not None:
        nperseg = window_sec * sf
    else:
        nperseg = (2 / low) * sf

    # Compute the modified periodogram (Welch)
    freqs, psd = welch(data, sf, nperseg=nperseg)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp


# calculating theta/alpha bandpower
win_sec=60
db = bandpower(df["EEG Fp1"], sf, [4,8], win_sec) / bandpower(df["EEG Fp1"], sf, [8,12], win_sec)

# Delta/beta ratio based on the relative power
db_rel = bandpower(df["EEG Fp1"], sf, [4,8], win_sec, True) / bandpower(df["EEG Fp1"], sf, [8,12], win_sec, True)

print('Theta/alpha ratio (absolute): %.6f' % db)
print('Theta/alpha ratio (relative): %.6f' % db_rel)


In [639]:
# importing data from saved CSV file consisting of PSDs of each channel of each subject
import pandas as pd


df1=pd.read_excel(r"C:\Users\Bhavya Prakash\OneDrive\Desktop\theta_alpha_ratio (WORKING STATE).xlsx")
print(df1)


# Calculate the average value of each row
row_averages = df1.mean(axis=1)

# Display the row averages
print(row_averages)
print(min(row_averages))
print(max(row_averages))


# distributing data to 5 arrays defining 5 mental workload levels i.e. 1,2,3,4,5.
arr1=[]
arr2=[]
arr3=[]
arr4=[]
arr5=[]

for i in row_averages:
    if 0.383703<=i<=0.7911246666666667:
        arr1.append(i)
    elif 0.7911246666666667<i<=1.1985463333333335:
        arr2.append(i)
    elif 1.1985463333333335<i<=1.6059680000000003:
        arr3.append(i)
    elif 1.6059680000000003<i<=2.013389666666667:
        arr4.append(i)
    elif 2.013389666666667<i<=2.4208113333333334:
        arr5.append(i)
    elif 2.4208113333333334<i<=2.8282330000000004:
        arr5.append(i)

print(len(arr1))
print(len(arr2))
print(len(arr3))
print(len(arr4))
print(len(arr5))

arr1.sort()
arr2.sort()
arr3.sort()
arr4.sort()
arr5.sort()


# importing data consisting of PSDs and Mental Workload levels.
df2=pd.read_excel(r"C:\Users\Bhavya Prakash\OneDrive\Desktop\theta_alpha_ratio (WORKING STATE).xlsx")
print(df2)


# Visualizing the data of a channel as an example
x=df2["EEG Fp1"][:25]
y=df2.Load[:25]
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.scatter(x,y)
plt.xlabel("uV")
plt.ylabel("Load")
plt.title("EEG Fp1")
plt.show()

,EEG Fp1,EEG Fp2,EEG F3,EEG F4,EEG F7,EEG F8,EEG Fz
0,0.643960,0.373429,0.492862,0.365592,0.561472,0.440456,0.364739
1,2.334138,1.595773,1.759945,1.581765,1.763199,1.806324,2.461548
2,0.951792,0.855790,0.890707,1.308588,0.565728,0.863658,1.602327
3,1.321261,0.935247,0.910835,0.673984,0.745914,0.978696,0.778270
4,0.835297,1.097330,1.185309,1.472631,1.200637,1.100765,1.523484
5,0.486106,0.513042,0.373281,0.420187,0.515135,0.313114,0.439621
6,1.211957,1.366215,1.156800,1.614809,1.280754,1.687613,1.871052
7,0.535506,1.206442,0.483269,0.747998,0.560588,0.728680,0.633370
8,1.718230,1.835368,1.885430,2.112142,1.714979,1.755307,2.011800
9,1.074406,0.680324,1.043714,0.982758,0.824776,1.841400,2.049896
